In [1]:
import pandas as pd

from utils.model import XGBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [5]:
with pd.HDFStore('./large_files/data.h5', mode='r') as store:
    df = store.get('feature_engineering/20240124')

In [6]:
categorical_columns = ['month'] + [c for c in df.columns if 'signal' in c]
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True, dtype=int)
df.info(verbose=3)

<class 'pandas.core.frame.DataFrame'>
Index: 2536959 entries, 0 to 2536958
Data columns (total 174 columns):
 #    Column               Dtype                           
---   ------               -----                           
 0    date                 object                          
 1    time                 object                          
 2    open                 float64                         
 3    high                 float64                         
 4    low                  float64                         
 5    close                float64                         
 6    volume               int64                           
 7    datetime             datetime64[ns, America/New_York]
 8    day                  object                          
 9    bb_high              float64                         
 10   bb_low               float64                         
 11   dollar_vol           float64                         
 12   z_close_5m           float64                 

In [8]:
df = df.set_index('datetime', drop=True)

# columns to drop: values that are not normalized
to_drop = [c for c in df.columns if 'sma' in c] + [c for c in df.columns if 'std' in c] + \
    ['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'day', 'dollar_vol']
df = df.drop(to_drop, axis=1)

# Wednesday

In [15]:
df2 = df[df.index.dayofweek == 2]
df2 = df2[df2.index.hour >= 9]
df2 = df2[df2.index.hour < 16]

df2.info(verbose=3, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 361820 entries, 2007-04-04 09:00:00-04:00 to 2023-11-22 15:59:00-05:00
Data columns (total 140 columns):
 #    Column               Non-Null Count   Dtype  
---   ------               --------------   -----  
 0    bb_high              361820 non-null  float64
 1    bb_low               361820 non-null  float64
 2    z_close_5m           361820 non-null  float64
 3    pct_close_5m         361820 non-null  float64
 4    z_volume_5m          361820 non-null  float64
 5    pct_volume_5m        361820 non-null  float64
 6    z_dollar_vol_5m      361820 non-null  float64
 7    pct_dollar_vol_5m    361820 non-null  float64
 8    dst_high_5m          361820 non-null  float64
 9    dst_low_5m           361820 non-null  float64
 10   dst_mean_5m          361820 non-null  float64
 11   dst_mean_high_5m     361820 non-null  float64
 12   dst_mean_low_5m      361820 non-null  float64
 13   rsi_5                361820 non-null  float64
 14   lag5